In [ ]:
!pip install quantstats

     |████████████████████████████████| 41 kB 333 kB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 6.4 MB 89.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import quantstats as qs

In [ ]:
class CrossAssetMomentum:
    def __init__(self, prices, lookback_period=10, holding_period=10, n_selection=5, cost=0.001, signal_method='dchannel', weightings='ew', 
                 long_only=False, short_only=False, show_analytics=True):
        self.returns = self.get_returns(prices)
        self.holding_returns = self.get_holding_returns(prices, holding_period)
        self.signal_method = signal_method  
        self.DchannelUpper, self.DchannelLower = self.Dchannel(prices, lookback_period)
        self.ma_slow, self.ma_fast = self.MA(prices, 100, 50)
        self.sharpe = self.Sharpe(prices, lookback_period)
        if signal_method == 'dchannel':
            self.signal = self.absolute_momentum(prices, lookback_period, long_only, short_only)
        elif signal_method == 'ma_crossover':
            self.signal = self.absolute_momentum(prices, lookback_period, long_only, short_only)
        elif signal_method == 'rmr':
            self.signal = self.relative_mean_reversion(prices, lookback_period, n_selection, long_only=long_only, short_only=short_only)
        elif signal_method == 'dm':
            self.signal = self.dual_momentum(prices, lookback_period, n_selection, long_only)
        if weightings == 'ew':
            self.cs_risk_weight = self.equal_weight(self.signal)
        elif weightings == 'emv':
            self.cs_risk_weight = self.equal_marginal_volatility(self.returns, self.signal)
        self.rebalance_weight = 1 / holding_period
        self.cost = self.transaction_cost(self.signal, cost)
        self.port_rets_wo_cash = self.backtest(self.holding_returns, self.signal, self.cost, 
                                               self.rebalance_weight, self.cs_risk_weight)
        
        self.ts_risk_weight = self.volatility_targeting(self.port_rets_wo_cash)
        self.port_rets = self.port_rets_wo_cash * self.ts_risk_weight
        if show_analytics == True:
            self.performance_analytics(self.port_rets_wo_cash)


    def MA(self, prices, slow, fast, ma_type ='SMA'):
        if ma_type == 'SMA':
          sma_slow = prices.rolling(slow).mean()
          sma_fast = prices.rolling(fast).mean()
          return sma_slow, sma_fast
        elif ma_type == 'EMA':
          ema_slow = prices.ewm(span=slow, adjust=False).mean()
          ema_fast = prices.ewm(span=fast, adjust=False).mean()
          return ema_slow, ema_fast


    def Dchannel(self, prices, lookback):
        dchannelUpper = prices.rolling(lookback).max().shift(1)
        dchannelLower = prices.rolling(lookback).min().shift(1)
        return dchannelUpper, dchannelLower


    def Sharpe(self, prices, lookback):
        std = prices.pct_change().rolling(lookback).std()
        mean = prices.pct_change().rolling(lookback).mean()
        sharpe = mean * np.sqrt(lookback) / std
        return sharpe
  

    def get_returns(self, prices):
        returns = prices.pct_change()
        return returns


    def get_holding_returns(self, prices, holding_period):
        holding_returns = prices.pct_change(periods=holding_period).shift(-holding_period)
        return holding_returns


    def long_pos_cal(self, x, y):
        position = np.where(x + y >= 1, 1, 0)
        return position


    def pos_cal(self, x, y):
        buy = np.logical_or(x + y >=1, y==1)
        sell = np.logical_or(x + y <=-1,y==-1)
        position1 = np.where(buy, 1, 0)
        position2 = np.where(sell, -1, 0)
        position = position1 + position2
        return position


    def absolute_momentum(self, prices, lookback, long_only, short_only):
        if self.signal_method == 'dchannel':
            buy = (prices > self.DchannelUpper).applymap(self.bool_converter)
            buy = buy.to_numpy()
            sell = -(prices < self.DchannelLower).applymap(self.bool_converter)
            sell = sell.to_numpy()
            buy_sell = buy + sell
            li = []
            if long_only == True:
                position = buy[0]
                li.append(position)
                for i in buy_sell[1:]:
                    position = self.long_pos_cal(position, i)
                    li.append(position)
                long_signal = pd.DataFrame(li, columns=prices.columns)
                long_signal.index = prices.index
                signal = long_signal
            else:
                position = buy_sell[0]
                li.append(position)
                for i in buy_sell[1:]:
                    position = self.pos_cal(position, i)
                    li.append(position)
                signal = pd.DataFrame(li, columns=prices.columns)
                signal.index = prices.index
        if self.signal_method == 'ma_crossover':
            long_signal = (self.ma_fast > self.ma_slow).applymap(self.bool_converter)
            short_signal = -(self.ma_fast < self.ma_slow).applymap(self.bool_converter)
            if long_only == True:
                signal = long_signal
            elif short_only == True:
                signal = short_signal
            else:
                signal = long_signal + short_signal
        return signal
        
    
    def relative_mean_reversion(self, prices, lookback, n_selection, long_only=False, short_only=False):
        returns = prices.pct_change(periods=lookback)
        rank = returns.rank(axis=1, ascending=False)
        long_signal = (rank >= len(rank.columns) - n_selection + 1).applymap(self.bool_converter)
        short_signal = -(rank <= n_selection).applymap(self.bool_converter)
        if long_only == True:
            signal = long_signal
        elif short_only == True:
            signal = short_signal
        else:
            signal = long_signal + short_signal
        return signal

    def relative_momentum(self, prices, lookback, n_selection, long_only=False):
        returns = prices.pct_change(periods=lookback)
        rank = returns.rank(axis=1, ascending=False)
        long_signal = (rank <= n_selection).applymap(self.bool_converter)
        short_signal = -(rank >= len(rank.columns) - n_selection + 1).applymap(self.bool_converter)
        if long_only == True:
            signal = long_signal
        else:
            signal = long_signal + short_signal
        return signal
    
    def dual_momentum(self, prices, lookback, n_selection, long_only=False):
        abs_signal = self.absolute_momentum(prices, lookback, long_only)
        rel_signal = self.relative_momentum(prices, lookback, n_selection, long_only)
        signal = (abs_signal == rel_signal).applymap(self.bool_converter) * abs_signal
        return signal

    def equal_weight(self, signal):
        total_signal = 1 / abs(signal).sum(axis=1)
        total_signal.replace([np.inf, -np.inf], 0, inplace=True)
        weight = pd.DataFrame(index=signal.index, columns=signal.columns).fillna(value=1)
        weight = weight.mul(total_signal, axis=0)
        return weight


    def equal_marginal_volatility(self, returns, signal):
        vol = (returns.rolling(14).std() * np.sqrt(14))
        vol_signal = vol * abs(signal)
        inv_vol = 1 / vol_signal
        inv_vol.replace([np.inf, -np.inf], 0, inplace=True)
        weight = inv_vol.div(inv_vol.sum(axis=1), axis=0)
        return weight


    def volatility_targeting(self, returns, target_vol=0.03):
        weight = target_vol / (returns.rolling(252).std() * np.sqrt(252))
        weight.replace([np.inf, -np.inf], 0, inplace=True)
        weight = weight.shift(1).fillna(0)
        return weight


    def transaction_cost(self, signal, cost=0.001):
        cost_df = (signal.diff() != 0).applymap(self.bool_converter) * cost
        cost_df.iloc[0] = 0
        return cost_df
    

    def backtest(self, returns, signal, cost, rebalance_weight, weighting):
        port_rets = ((signal * returns - cost) * rebalance_weight * weighting).sum(axis=1)
        return port_rets


    def performance_analytics(self, returns):
        qs.reports.html(returns, output='./file-name.html')


    def bool_converter(self, bool_var):
        if bool_var == True:
            result = 1
        elif bool_var == False:
            result = 0
        return result


def get_price_df(url):
    df = pd.read_csv(url).dropna()
    df.index = pd.to_datetime(df['Date'])
    df = df.drop(columns=['Date'])
    return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
prices = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/크립토 프로젝트/close_price_4h_1000_0522_future.xlsx', index_col=0)
momentum = CrossAssetMomentum(prices)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
